# 🎵 MOSS: Multi-Objective Sound Synthesis

## Hledání Pareto-optimálních směsí spektrogramů

Tento notebook slouží jako referenční implementace a soběstačná demonstrace systému MOSS. Algoritmy zde přesně odpovídají webovému frontendu na https://moss.app.kuchar.dpdns.org.

### 🎯 Definice úlohy

Nacházíme se v doméně vícekriteriální optimalizace (Multi-Objective Optimization). Naším cílem je nalézt zobrazení, které kóduje sémantickou informaci z vizuální domény do domény akustické, přičemž minimalizuje ztrátu informace v obou modalitách.

Nechť $I$ je cílový obrázek (např. Mona Lisa) a $A$ je cílový zvuk (např. Čajkovský). Hledáme matici spektrogramu $S$, která minimalizuje vektorovou chybovou funkci:
$$\min_{S} F(S) = [L_\mathrm{visual}(S, I), L_\mathrm{audio}(S, A)]^T$$

Tento problém nemá jediné globální řešení, nýbrž množinu optimálních kompromisů, kterou nazýváme Paretovou frontou. Řešení $S^*$ je Pareto-optimální, pokud neexistuje jiné řešení $S'$, pro které by platilo:
$$\forall i: L_i(S') \leq L_i(S^*) \wedge \exists j: L_j(S') < L_j(S^*)$$

To jest, nemůžeme vylepšit vizuální podobnost, aniž bychom poškodili zvukovou věrnost, a naopak.

### 🧠 Klíčové koncepty a architektura

**Spektrální maskování:**
Namísto přímé syntézy pixelů optimalizujeme skalární pole (masku) $M \in [0,1]^{F \times T}$, které řídí lineární interpolaci mezi magnitudami zdrojových signálů.

**Problém spektrální rekonstrukce:**
Lidský sluch je extrémně citlivý na časovou strukturu signálu, která je v STFT reprezentaci zakódována ve fázi. Vizuální rekonstrukce však operuje pouze nad amplitudou (magnitudou). Pro zachování srozumitelnosti audia fixujeme fázi cílového zvuku ($\phi_\mathrm{audio}$) a optimalizujeme pouze magnitudu. Výsledný signál je rekonstruován jako $x(t) = \text{ISTFT}(|S_\mathrm{mixed}| \cdot e^{j\phi_\mathrm{audio}})$.

**Hybridní optimalizace:**
Pro efektivní mapování stavového prostoru využíváme dvoufázový přístup:
1. **Gradientní single-objective optimalizace (ADAM)** pro rychlou konvergenci v určitých směrech.
2. **Evoluční algoritmus (NSGA-II)** pro globální prohledávání a zmapování Paretovy fronty.

## 1. Základní konfigurace a Signálové zpracování

Zde definujeme parametry pro Short-Time Fourier Transform (STFT). Volba těchto konstant je kritická pro vyvážení Gaborova limitu, který říká, že nelze mít libovolně vysokou přesnost v čase i ve frekvenci současně:

$$\sigma_t \cdot \sigma_\omega \geq \frac{1}{2}$$

* **SAMPLE_RATE = 16000**: Nyquistova frekvence je 8 kHz, což pokrývá většinu sémanticky podstatného spektra pro lidskou řeč a hudbu.
* **N_FFT = 1024**: Velikost okna. Určuje frekvenční rozlišení na $\approx 15.6$ Hz na bin.
* **HOP_LENGTH = 256**: Posun okna (75% překryv). Zajišťuje hladkou časovou rekonstrukci a minimalizuje 'spectral leakage' artefakty.

Pro urychlení výpočtů zavádíme koncept **Proxy Optimalizace**. Optimalizační smyčka běží v redukovaném rozlišení (`PROXY_HEIGHT = 129`), což odpovídá čtvrtinovému vzorkování frekvenční osy. Výsledek je následně upsamplován bilineární interpolací.

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import HTML, Audio, display
import warnings
warnings.filterwarnings('ignore')

SAMPLE_RATE = 16000
N_FFT = 1024
HOP_LENGTH = 256
WIN_LENGTH = N_FFT
GRID_HEIGHT = 64
GRID_WIDTH = 128
N_PARAMS = GRID_HEIGHT * GRID_WIDTH

# Pro zrychlení provádíme proxy optimalizaci v nižším rozlišení
PROXY_HEIGHT = 129
DEVICE = 'cpu'

# Parametry optimalizací, stopping kriterium je všude počet iterací
SINGLE_STEPS = 500
PARETO_SEED_STEPS = 200
PARETO_GENERATIONS = 50
PARETO_SEED_POP = 10
PARETO_EVOL_POP = 100

def plot_spectrogram(ax, mag_tensor, title=None, show_colorbar=False):
    
    spec_db = 20 * torch.log10(mag_tensor + 1e-8).cpu().numpy()
    ref_max = np.percentile(spec_db, 99.5)
    vmin = ref_max - 80
    vmax = ref_max
    
    im = ax.imshow(spec_db, origin='lower', aspect='auto', cmap='magma',
                   vmin=vmin, vmax=vmax)
    if title:
        ax.set_title(title)
    if show_colorbar:
        plt.colorbar(im, ax=ax, label='dB')
    return im

print(f"PyTorch verze: {torch.__version__}")
print(f"Zařízení (hard-coded na CPU pro kompatibilitu): {DEVICE}")
print(f"Prostor parametrů: {GRID_HEIGHT}×{GRID_WIDTH} = {N_PARAMS} parametrů")


## 2. Definice Ztrátových funkcí

Aby optimalizátor "viděl" a "slyšel", musíme rozumně definovat metriky vzdálenosti v obou doménách.

### 👁️ Visual Loss: Mean Absolute Error (MAE)

Pro vizuální složku používáme $L_1$ normu. Na rozdíl od MSE ($L_2$) je $L_1$ méně citlivá na odlehlé hodnoty (outliers) a produkuje ostřejší hrany ve spektrogramu, což je žádoucí pro rozeznatelnost obrazu.

$$\mathcal{L}_\mathrm{visual} = ||M_\mathrm{mixed} - M_\mathrm{image}||_1 = \frac{1}{N} \sum_{f,t} |M_\mathrm{mixed}(f,t) - M_\mathrm{image}(f,t)|$$

### 👂 Audio Loss: Log-Spectral Distance

Pro audio složku nemůžeme použít lineární vzdálenost, protože lidské vnímání hlasitosti je přibližně logaritmické (v souladu s Weberovým-Fechnerovým zákonem). Minimalizujeme tedy vzdálenost v logaritmické doméně:

$$\mathcal{L}_\mathrm{audio} = ||\log(M_\mathrm{mixed} + \epsilon) - \log(M_\mathrm{audio} + \epsilon)||_1$$

kde $\epsilon = 10^{-8}$ zajišťuje numerickou stabilitu.

### ⚖️ Normalizace dle Nadiru

Abychom mohli tyto dvě kritéria porovnávat, normalizujeme je do intervalu [0,1] pomocí odhadu Nadiru, body loss funkce škálujeme pomocí:

*   $\mathrm{Scale}_\mathrm{vis} = (\mathcal{L}_\mathrm{vis}(A, I))^{-1}$ (Obrázková loss funkce čistého zvuku)
*   $\mathrm{Scale}_\mathrm{aud} = (\mathcal{L}_\mathrm{aud}(I, A))^{-1}$ (Zvuková loss funkce čistého obrázku)

In [ ]:
def calc_audio_mag_loss(mixed_mag, target_audio_mag):

    mixed_log = torch.log(mixed_mag + 1e-8)
    target_log = torch.log(target_audio_mag + 1e-8)
    target_log = target_log.expand_as(mixed_log)
    
    loss = F.l1_loss(mixed_log, target_log, reduction='none')
    
    if mixed_mag.dim() == 3:
        return loss.mean(dim=(1, 2))
    return loss.mean()


def calc_visual_loss(mixed_mag, target_image_mag):

    diff = torch.abs(mixed_mag - target_image_mag)
    
    if mixed_mag.dim() == 3:
        return diff.mean(dim=(1, 2))
    return diff.mean()

## 3. Topologie masky a Gaussovské vyhlazení

Optimalizujeme matici parametrů $\theta \in \mathbb{R}^{H \times W}$. Abychom zabránili vzniku ostrých spektrálních diskontinuit, které by se projevily jako zvukové a obrazové artefakty, aplikujeme na masku konvoluci s Gaussovským jádrem.

Proces transformace parametrů na masku je
$$\text{mask} = \sigma(\text{Upsample}(\theta * G_\sigma)),$$

kde $G_\sigma$ je 2D Gaussovské jádro
$$G_\sigma(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}.$$

Tato operace funguje jako regularizace, která vynucuje prostorovou korelaci parametrů, jednoduše řečeno 'vyhlazuje'; vyšší $\sigma$ vede k "hladšímu" zvuku a abstraktnějšímu obrazu.

In [ ]:
def gaussian_blur_2d(x, sigma=1.0):

    if sigma < 0.5:
        return x
    
    kernel_size = int(6 * sigma) | 1
    kernel_size = max(3, kernel_size)
    
    coords = torch.arange(kernel_size, dtype=x.dtype, device=x.device) - kernel_size // 2
    g = torch.exp(-coords**2 / (2 * sigma**2))
    g = g / g.sum()
    
    pad = kernel_size // 2
    original_dim = x.dim()
    
    if x.dim() == 2:
        x = x.unsqueeze(0).unsqueeze(0)
    elif x.dim() == 3:
        x = x.unsqueeze(1)
    
    x = F.conv2d(F.pad(x, (pad, pad, 0, 0), mode='replicate'), 
                 g.view(1, 1, 1, -1), padding=0)
    x = F.conv2d(F.pad(x, (0, 0, pad, pad), mode='replicate'), 
                 g.view(1, 1, -1, 1), padding=0)
    
    if original_dim == 2:
        return x.squeeze(0).squeeze(0)
    elif original_dim == 3:
        return x.squeeze(1)
    return x

## 4. Architektura Enkodéru (The Mask Encoder)

Třída `MaskEncoder` je srdcem celého systému. Zajišťuje diferenciabilní průchod z prostoru parametrů masky do prostoru zvuku.

### Gain Staging a Dynamický Rozsah

Jedním z technicky nejnáročnějších aspektů je sladění dynamického rozsahu obrazu a zvuku.
- Spektrogramy audia mají obrovský dynamický rozsah (často > 80 dB).
- Obrázky (v rozsahu 0–255) jsou lineární a mají nízký kontrast.

Implementujeme adaptivní **Gain Staging**, který mapuje jas obrazu do decibelové škály zvuku. Algoritmus vypočítá „strop“ (ceiling) a „podlahu“ (noise floor) cílového zvuku a lineárně transformuje normalizovaný obraz $I_{norm} \in [0,1]$ do log-magnitudy zvuku:

$$\log(M_{image}) = I_{norm} \cdot (dB_{max} - dB_{min}) + dB_{min}$$

Tím zajistíme, že černá barva odpovídá tichu (noise floor) a bílá maximální hlasitosti, čímž maximalizujeme perceptibilitu obrazu ve spektrogramu zvuku.

### Rekonstrukce signálu

Jak bylo zmíněno v úvodu, pro převod zpět do časové domény (waveform) používáme inverzní STFT.

In [ ]:
class MaskProcessor(nn.Module):
    """Zpracovává parametry masky aplikací Gaussova rozmazání a interpolace."""

    def __init__(self, h, w, sigma, device):
        super().__init__()
        self.h = h
        self.w = w
        self.device = device
        self._init_gaussian_kernel(sigma)

    def _init_gaussian_kernel(self, sigma):
        """Inicializuje Gaussovo jádro pro vyhlazení masky."""
        kernel_size = int(2 * math.ceil(2 * sigma) + 1)
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()
        xy_grid = torch.stack([x_grid, y_grid], dim=-1).float()

        mean = (kernel_size - 1) / 2.0
        variance = sigma**2.0
        gaussian_kernel = (1.0 / (2.0 * math.pi * variance)) * torch.exp(
            -torch.sum((xy_grid - mean) ** 2.0, dim=-1) / (2 * variance)
        )
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        self.blur_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size).to(
            self.device
        )
        self.kernel_padding = kernel_size // 2

    def forward(self, params, target_h, target_w):
        """Aplikuje rozmazání na parametry a změní jejich velikost na cílové rozlišení."""
        B = params.shape[0]
        grid = params.view(B, 1, self.h, self.w)

        # Aplikace konvoluce pro vyhlazení hran masky
        grid_blurred = F.conv2d(grid, self.blur_kernel, padding=self.kernel_padding)

        # Interpolace na cílovou velikost spektrogramu
        mask = F.interpolate(
            grid_blurred,
            size=(target_h, target_w),
            mode="bilinear",
            align_corners=False,
        )

        return mask.squeeze(1)


class MaskEncoder(nn.Module):
    """Kóduje parametry do audia pomocí prolínání spektrogramů založeného na masce."""

    def __init__(
        self,
        target_image: torch.Tensor,
        target_audio_path: str,
        grid_height: int = 128,
        grid_width: int = 256,
        smoothing_sigma: float = 1.0,
        device: str = DEVICE,
    ):
        super().__init__()
        self.device = device
        self.grid_height = grid_height
        self.grid_width = grid_width
        self.n_params = grid_height * grid_width
        self.smoothing_sigma = smoothing_sigma

        # Načtení a příprava referenčního audia
        audio, sr = torchaudio.load(target_audio_path)
        if sr != SAMPLE_RATE:
            audio = torchaudio.functional.resample(audio, sr, SAMPLE_RATE)

        audio = audio.mean(dim=0, keepdim=True)  # Převod na mono

        self.mask_processor = MaskProcessor(
            grid_height, grid_width, smoothing_sigma, device
        )

        self.register_buffer("target_audio_waveform", audio)

        # Výpočet STFT (Krátkodobá Fourierova transformace)
        window = torch.hann_window(N_FFT).to(device)
        stft = torch.stft(
            audio.to(device),
            n_fft=N_FFT,
            hop_length=HOP_LENGTH,
            win_length=WIN_LENGTH,
            window=window,
            return_complex=True,
        )

        self.audio_mag = stft.abs() + 1e-8
        self.audio_phase = stft.angle()

        self.full_height = self.audio_mag.shape[1]
        self.full_width = self.audio_mag.shape[2]

        self.audio_log = torch.log(self.audio_mag)

        # Příprava cílového obrázku pro spektrogram
        img = target_image.to(device)
        if img.dim() == 3:
            img = img.unsqueeze(0)

        target_h_visual = self.full_height
        img_flipped = torch.flip(img, dims=[-2]) # Překlopení pro správnou orientaci frekvencí
        img_full_freq = F.interpolate(
            img_flipped,
            size=(target_h_visual, self.full_width),
            mode="bilinear",
            align_corners=False,
        )
        img_resized = img_full_freq

        # Dynamické řízení zisku (Gain Staging) pro sladění úrovní obrazu a audia
        audio_log_max = torch.quantile(self.audio_log, 0.98)
        audio_max_val = self.audio_log.max()
        audio_floor_val = torch.quantile(self.audio_log, 0.01)

        target_ceiling = audio_max_val - 0.5
        headroom_nat = (audio_log_max - target_ceiling).item()
        dynamic_range_nat = (target_ceiling - audio_floor_val).item() + 0.5
        dynamic_range_nat = max(4.0, min(dynamic_range_nat, 12.0))

        print("Dynamické řízení zisku:")
        print(f"  > Max. audio: {audio_max_val:.2f}, Práh (q01): {audio_floor_val:.2f}")
        print(f"  > Cílový strop: {target_ceiling:.2f}")
        print(f"  > Adaptivní dynamický rozsah: {dynamic_range_nat:.2f}")

        audio_log_ceil = audio_log_max - headroom_nat
        audio_log_floor = audio_log_ceil - dynamic_range_nat

        self.audio_log = torch.clamp(self.audio_log, min=audio_log_floor)

        # Normalizace a úprava kontrastu obrazu
        img_01 = img_resized.squeeze(0)
        img_01 = (img_01 - img_01.min()) / (img_01.max() - img_01.min() + 1e-8)
        img_01 = img_01.pow(1.8)

        # Mapování obrazu do logaritmické domény magnitudy audia
        self.image_log = img_01 * (audio_log_ceil - audio_log_floor) + audio_log_floor
        self.image_mag = torch.exp(self.image_log)
        self.audio_mag_static = torch.exp(self.audio_log)

        # Nastavení proxy rozlišení pro rychlejší náhledy
        self.proxy_height = 129
        self.proxy_width = self.full_width // 2

        self.image_mag_proxy = F.interpolate(
            self.image_mag.unsqueeze(0),
            size=(self.proxy_height, self.proxy_width),
            mode="bilinear",
            align_corners=False,
        ).squeeze(0)

        self.audio_mag_proxy = F.interpolate(
            self.audio_mag_static.unsqueeze(0),
            size=(self.proxy_height, self.proxy_width),
            mode="bilinear",
            align_corners=False,
        ).squeeze(0)

        self.image_mag_ref = self.image_mag_proxy
        self.audio_mag = self.audio_mag_proxy

        self.image_mag_full = self.image_mag
        self.audio_mag_full = self.audio_mag_static

        print(f"Spektrogram: {self.full_height}×{self.full_width}")
        print(f"Proxy: {self.proxy_height}×{self.proxy_width}")

    def _compute_spectrogram(self, mask, img_mag, aud_mag):
        """Lineárně interpoluje mezi magnitudou obrazu a audia pomocí masky."""
        return mask * img_mag + (1 - mask) * aud_mag

    def _reconstruct_audio(self, mixed_mag, phase):
        """Rekonstruuje audio signál z magnitudy a fáze pomocí inverzní STFT."""
        complex_stft = torch.polar(mixed_mag, phase)

        audio_recon = torch.istft(
            complex_stft,
            n_fft=N_FFT,
            hop_length=HOP_LENGTH,
            win_length=WIN_LENGTH,
            window=torch.hann_window(N_FFT, device=mixed_mag.device),
        )

        # Normalizace hlasitosti
        max_val = audio_recon.abs().max(dim=-1, keepdim=True)[0].clamp(min=1e-8)
        audio_recon = audio_recon / max_val * 0.9
        return audio_recon

    def forward(self, params: torch.Tensor, return_wav: bool = True):
        """Hlavní dopředný průchod: generování masky, smíchání a volitelná rekonstrukce audia."""
        batch_size = params.shape[0]

        target_h = self.proxy_height if not return_wav else self.full_height
        target_w = self.proxy_width if not return_wav else self.full_width

        mask = self.mask_processor(params, target_h, target_w)

        if not return_wav:
            img_mag = self.image_mag_ref.expand(batch_size, -1, -1)
            aud_mag = self.audio_mag.expand(batch_size, -1, -1)
        else:
            img_mag = self.image_mag_full.expand(batch_size, -1, -1)
            aud_mag = self.audio_mag_full.expand(batch_size, -1, -1)

        phase = self.audio_phase.expand(batch_size, -1, -1) if return_wav else None

        mixed_mag = self._compute_spectrogram(mask, img_mag, aud_mag)

        audio_recon = None
        if return_wav:
            audio_recon = self._reconstruct_audio(mixed_mag, phase)

        return audio_recon, mixed_mag

## 5. Příprava dat
Načítáme vstupní modality.

**Audio**: Je převedeno na mono a převzorkováno na 16kHz.

**Obraz**: Je převeden na stupně šedi (Luma), oříznut a interpolován tak, aby jeho výška odpovídala Nyquistově frekvenci v binech ($NFFT/2+1=513$).

Parametr `sigma=5.0` byl zvolen empiricky.

In [ ]:
img_path = 'monalisa.jpg'
img_pil = Image.open(img_path).convert('L')
img_tensor = torch.from_numpy(np.array(img_pil)).float() / 255.0
img_tensor = img_tensor.unsqueeze(0)

audio_path = 'tchaikovsky.mp3'
waveform, sr = torchaudio.load(audio_path)
duration_sec = waveform.shape[-1] / sr
raw_width = int(duration_sec * 4.0)
grid_width = ((raw_width + 15) // 16) * 16
if grid_width < 16:
    grid_width = 16
grid_height = 64

sigma = 5.0
encoder = MaskEncoder(
    img_tensor, 
    audio_path,
    grid_height=grid_height,
    grid_width=grid_width,
    smoothing_sigma=sigma,
    device=DEVICE
)

GRID_HEIGHT = grid_height
GRID_WIDTH = grid_width
N_PARAMS = GRID_HEIGHT * GRID_WIDTH

print(f"\nGrid: {GRID_HEIGHT}×{GRID_WIDTH} = {N_PARAMS} parameters")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(img_pil, cmap='gray')
axes[0].set_title('Cílový obrázek: Mona Lisa', fontsize=12)
axes[0].axis('off')

plot_spectrogram(axes[1], encoder.image_mag_full[0], 'Spektrogramová Mona Lisa')
axes[1].set_xlabel('Čas (snímky)')
axes[1].set_ylabel('Frekvence (biny)')

plot_spectrogram(axes[2], encoder.audio_mag_full[0], 'Cílový zvuk: Čajkovský')
axes[2].set_xlabel('Čas (snímky)')

plt.tight_layout()
plt.show()

print("🔊 Originální Čajkovský:")
display(Audio(encoder.target_audio_waveform[0].numpy(), rate=SAMPLE_RATE))

## 6. Správce Paretovy optimalizace (Pareto Manager)

Tato třída implementuje logiku vektorové optimalizace. Místo hledání jednoho bodu spravujeme celou populaci řešení.

### Skalarizace cílové funkce

Gradientní metody (jako ADAM) vyžadují skalární (jednorozměrnou) ztrátovou funkci. Pro vytvoření populace rozprostřené podél Paretovy fronty používáme metodu vážených součtů (Weighted Sum Method).

Každý jedinec $i$ v populaci optimalizuje unikátní kombinaci vah $w_\mathrm{vis}^{(i)}, w_\mathrm{aud}^{(i)}$, pro jeho loss funkci platí

$$\mathcal{L}_\mathrm{total}^{(i)} = \gamma \cdot w_\mathrm{vis}^{(i)} \cdot \hat{\mathcal{L}}_\mathrm{vis} + w_\mathrm{aud}^{(i)} \cdot \hat{\mathcal{L}}_\mathrm{aud},$$

kde $\gamma = 20.0$ je empirický váhový faktor. Zjistili jsme, že audio ztráta je přes logaritmické škálování v jistém smyslu "tvrdohlavější" než vizuální ztráta. Bez tohoto faktoru by optimalizace téměř vždy konvergovala k čistému zvuku.

Pro kompenzaci nelineárního vztahu mezi poměrem vah a percepcí spektrogramu dále v rámci optimalizace užíváme mocninné rozdělení vah, které vede k lepšímu pokrytí objective space.

In [ ]:
class ParetoManager(nn.Module):
    """
    Spravuje populaci masek pro optimalizaci.
    """
    
    def __init__(self, encoder, pop_size=10, learning_rate=0.05):
        super().__init__()
        self.encoder = encoder
        self.pop_size = pop_size
        self.grid_h = encoder.grid_height
        self.grid_w = encoder.grid_width
        
        # Inicializace populace v prostoru logitů (neomezené, sigmoid na [0,1])
        self.mask_logits = nn.Parameter(
            torch.randn(pop_size, self.grid_h * self.grid_w) * 0.5
        )
        
        # ====================================================================
        # Mocninné rozdělení vah (kompenzuje 20x vizuální posílení)
        # ====================================================================
        alpha = torch.linspace(0, 1, pop_size)
        self.weights_img = (1.0 - alpha).pow(4.0)
        self.weights_aud = 1.0 - self.weights_img
        
        # Normalizace na součet 1
        total = self.weights_img + self.weights_aud
        self.weights_img = self.weights_img / total
        self.weights_aud = self.weights_aud / total
        
        # Vynucení extrémů: čistý obraz a čisté audio
        if pop_size >= 2:
            with torch.no_grad():
                self.mask_logits[0].fill_(10.0)   # Index 0: Čistý obraz
                self.mask_logits[-1].fill_(-10.0)  # Index -1: Čisté audio
        
        # Inicializace ADAMa
        self.optimizer = optim.Adam([self.mask_logits], lr=learning_rate)
        
        # Normalizační faktory (budou vypočteny)
        self.scale_vis = 1.0
        self.scale_aud = 1.0
        
    def calculate_normalization(self):
        """
        Výpočet nejhorších ztrát pro normalizaci cílů do rozsahu [0, 1].
        """
        print("Vypočítávám normalizační faktory...")
        with torch.no_grad():
            # Nejhorší vizuál: čisté audio (maska = 0)
            max_vis_loss = torch.abs(
                self.encoder.audio_mag - self.encoder.image_mag_ref
            ).mean().item()
            
            # Nejhorší audio: čistý obraz (maska = 1)
            max_aud_loss = calc_audio_mag_loss(
                self.encoder.image_mag_ref, 
                self.encoder.audio_mag
            ).item()
            
            # Zabránění dělení nulou
            self.scale_vis = 1.0 / max(max_vis_loss, 1e-6)
            self.scale_aud = 1.0 / max(max_aud_loss, 1e-6)
            
            print(f"  Max vizuální ztráta: {max_vis_loss:.4f} → měřítko: {self.scale_vis:.4f}")
            print(f"  Max audio ztráta:    {max_aud_loss:.4f} → měřítko: {self.scale_aud:.4f}")
    
    def optimize_step(self):
        """
        Provede jeden krok gradientního sestupu pro všechny členy populace.
                
        Returns:
            avg_vis: Průměrná normalizovaná vizuální ztráta
            avg_aud: Průměrná normalizovaná audio ztráta
        """
        self.optimizer.zero_grad()
        
        # Získání aktuálních masek
        masks = torch.sigmoid(self.mask_logits)
        
        # Dopředný průchod - return_wav=False automaticky používá PROXY!
        _, mixed_mag = self.encoder(masks, return_wav=False)
        
        # Výpočet ztrát (odpovídá backendu - používá proxy reference)
        diff = torch.abs(mixed_mag - self.encoder.image_mag_ref)
        raw_loss_vis = diff.mean(dim=(1, 2))
        raw_loss_aud = calc_audio_mag_loss(mixed_mag, self.encoder.audio_mag)
        
        # Normalizace
        loss_vis = raw_loss_vis * self.scale_vis
        loss_aud = raw_loss_aud * self.scale_aud
        
        # Skalarizovaná ztráta s 20x vizuálním posílením
        total_loss = torch.sum(
            self.weights_img * loss_vis * 20.0 + self.weights_aud * loss_aud
        )
        
        # Zpětný průchod a krok optimalizátoru
        total_loss.backward()
        self.optimizer.step()
        
        # Vynucení omezení kotev
        if self.pop_size >= 2:
            with torch.no_grad():
                self.mask_logits[0].fill_(20.0)
                self.mask_logits[-1].fill_(-20.0)
        
        return loss_vis.detach().mean().item(), loss_aud.detach().mean().item()
    
    def get_current_front(self):
        """Získání aktuálních ztrát populace pro vykreslování."""
        with torch.no_grad():
            masks = torch.sigmoid(self.mask_logits)
            _, mixed_mag = self.encoder(masks, return_wav=False)
            
            diff = torch.abs(mixed_mag - self.encoder.image_mag_ref)
            loss_vis = diff.mean(dim=(1, 2)) * self.scale_vis
            loss_aud = calc_audio_mag_loss(mixed_mag, self.encoder.audio_mag) * self.scale_aud
            
            return loss_vis.numpy(), loss_aud.numpy()

## 7. Jedno-kriteriální Optimalizace (Gradient Descent)
Zde provádíme základní "sondáž" prostoru řešení pro jednotlivé pareto-optimální body.

Algoritmus ADAM (Adaptive Moment Estimation) zde funguje jako lokální vyhledávač. Díky tomu, že máme analyticky definované gradienty pro všechny operace (včetně STFT a interpolací), můžeme efektivně najít optimum pro danou váhovou kombinaci.

Sledujme vývoj ztrátových funkcí v čase. Všimněme si, že vizuální a audio ztráty jsou dle předpokladu v antagonistickém vztahu a pokles jedné koreluje s nárůstem druhé.

In [ ]:
def single_objective_optimize(encoder, w_vis=0.5, w_aud=0.5, steps=SINGLE_STEPS, lr=0.05):
    """
    Jednokriteriální optimalizace se zadanými vahami.
    
    Toto přesně odpovídá frontendu pro tři předvolby:
    - "Lepší obraz": w_vis=0.8, w_aud=0.2
    - "Vyvážené":    w_vis=0.5, w_aud=0.5
    - "Lepší zvuk":  w_vis=0.2, w_aud=0.8
    
    Argumenty:
        encoder: Instance MaskEncoder
        w_vis: Vizuální váha (0-1)
        w_aud: Audio váha (0-1)
        steps: Počet optimalizačních kroků (výchozí: 500)
        lr: Rychlost učení (learning rate)
    
    Návratové hodnoty:
        mask: Tenxor optimalizované masky
        history: Seznam n-tic (vis_loss, aud_loss)
    """
    # Vytvoření manažera pro jednoho jedince
    manager = ParetoManager(encoder, pop_size=1, learning_rate=lr)
    manager.calculate_normalization()
    
    # Přepsání vah (normalizace na součet 1)
    total_weight = w_vis + w_aud
    w_vis_norm = w_vis / total_weight
    w_aud_norm = w_aud / total_weight
    
    manager.weights_img = torch.tensor([w_vis_norm])
    manager.weights_aud = torch.tensor([w_aud_norm])
    
    # Inicializace masky na neutrální hodnotu (logit = 0 → sigmoid = 0.5)
    with torch.no_grad():
        manager.mask_logits.data.zero_()
    
    history = []
    
    print(f"Optimalizace s vahami: Vizuální={w_vis_norm:.2f}, Audio={w_aud_norm:.2f}")
    print(f"Kroky: {steps}, Rychlost učení: {lr}")
    
    for step in range(1, steps + 1):
        avg_vis, avg_aud = manager.optimize_step()
        history.append((avg_vis, avg_aud))
        
        if step % 100 == 0 or step == 1:
            print(f"  Krok {step:4d}: Vizuální={avg_vis:.4f}, Audio={avg_aud:.4f}")
    
    # Získání finální masky
    final_mask = torch.sigmoid(manager.mask_logits).detach()
    
    return final_mask, history

# Spustit tři předvolby frontendu
import time

print("=" * 60)
print("🎨 PRIORITA OBRAZU (80/20)")
print("=" * 60)
start = time.time()
mask_visual, hist_visual = single_objective_optimize(encoder, w_vis=0.8, w_aud=0.2)
print(f"Dokončeno za {time.time() - start:.1f}s\n")

print("=" * 60)
print("⚖️ VYVÁŽENÉ (50/50)")
print("=" * 60)
start = time.time()
mask_balanced, hist_balanced = single_objective_optimize(encoder, w_vis=0.5, w_aud=0.5)
print(f"Dokončeno za {time.time() - start:.1f}s\n")

print("=" * 60)
print("🔊 PRIORITA ZVUKU (20/80)")
print("=" * 60)
start = time.time()
mask_audio, hist_audio = single_objective_optimize(encoder, w_vis=0.2, w_aud=0.8)
print(f"Dokončeno za {time.time() - start:.1f}s\n")

# Vykreslení konvergence
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, hist, title, color in [
    (axes[0], hist_visual, 'Vizuální priorita (80/20)', 'purple'),
    (axes[1], hist_balanced, 'Vyvážené (50/50)', 'gray'),
    (axes[2], hist_audio, 'Audio priorita (20/80)', 'green'),
]:
    vis_losses = [h[0] for h in hist]
    aud_losses = [h[1] for h in hist]
    
    ax.plot(vis_losses, label='Vizuální ztráta', color='purple', alpha=0.8)
    ax.plot(aud_losses, label='Audio ztráta', color='green', alpha=0.8)
    ax.set_xlabel('Krok')
    ax.set_ylabel('Normalizovaná ztráta')
    ax.set_title(title)
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Vizualizace výsledků
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

results = [
    (mask_visual, 'Vizuální priorita (80/20)'),
    (mask_balanced, 'Vyvážené (50/50)'),
    (mask_audio, 'Audio priorita (20/80)')
]

for i, (mask, title) in enumerate(results):
    # Generování výstupu v plném rozlišení
    _, mixed_mag = encoder(mask, return_wav=True)  # return_wav=True = plné rozlišení
    
    # Vizualizace masky
    axes[0, i].imshow(
        mask.view(GRID_HEIGHT, GRID_WIDTH).numpy(), 
        cmap='gray', vmin=0, vmax=1
    )
    axes[0, i].set_title(f'{title}\nMaska (bílá=obraz, černá=audio)')
    axes[0, i].axis('off')
    
    # Spektrogram (s měřítkem odpovídajícím frontendu)
    plot_spectrogram(axes[1, i], mixed_mag[0], 'Výsledný spektrogram')
    axes[1, i].set_xlabel('Čas')
    axes[1, i].set_ylabel('Frekvence')

plt.tight_layout()
plt.show()

# Poslechněte si výsledky

print("🎨 PRIORITA OBRAZU (80/20):")
wav_visual, _ = encoder(mask_visual, return_wav=True)
display(Audio(wav_visual[0].numpy(), rate=SAMPLE_RATE))

print("⚖️  VYVÁŽENÉ ŘEŠENÍ (50/50):")
wav_balanced, _ = encoder(mask_balanced, return_wav=True)
display(Audio(wav_balanced[0].numpy(), rate=SAMPLE_RATE))

print("🔊 PRIORITA ZVUKU (20/80):")
wav_audio, _ = encoder(mask_audio, return_wav=True)
display(Audio(wav_audio[0].numpy(), rate=SAMPLE_RATE))

## 8. Mapování Kompletní Paretovy Fronty (NSGA-II)

Pro efektivní zmapování bodů na Paretově frontě nasazujeme hybridní optimalizaci.

### Fáze 1: Gradientní Seeding

Nejprve spustíme krátkou, intenzivní dávku paralelních gradientních sestupů s různými vahami. Tím nalezneme několik jedinců blízko Paretovy fronty. Tyto body slouží jako vysoce kvalitní genetický materiál (Seeds) pro další fázi.

### Fáze 2: Evoluční Expanze (NSGA-II)

Přecházíme na algoritmus NSGA-II (Non-dominated Sorting Genetic Algorithm II) [Deb et al., 2002]. Tento algoritmus nepracuje s gradienty, ale s přímým porovnáváním fitness vektorů.

**Klíčové mechaniky:**

*   **Non-dominated Sorting:** Řadí populaci do "vrstev" podle toho, kolika jinými řešeními je daný jedinec dominován. První vrstva je aktuální odhad Paretovy fronty.
*   **Crowding Distance:** Metrika, která preferuje řešení v méně prozkoumaných oblastech fronty. To brání předčasné konvergenci do jednoho bodu a zajišťuje biodiverzitu populace a tedy také pokrytí celé fronty.
*   **SBX Crossover & Polynomial Mutation:** Genetické operátory simulující křížení a mutaci reálných chromozomů, aplikované na parametry masky.

Tento proces postupně "vyhlazuje" a zahušťuje mezeru mezi původními gradientními body, čímž odhaluje strukturu Paretovy fronty.

In [ ]:
def run_pareto_optimization(encoder, 
                            seed_steps=PARETO_SEED_STEPS,
                            seed_pop=PARETO_SEED_POP,
                            n_gen=PARETO_GENERATIONS,
                            evol_pop=PARETO_EVOL_POP):
    """
    Kompletní optimalizace Paretovy hranice přesně odpovídající frontendu.
    
    Argumenty:
        encoder: Instance MaskEncoder
        seed_steps: Kroky gradientního nasazování (výchozí: 200)
        seed_pop: Velikost počáteční populace (výchozí: 10)
        n_gen: Počet generací NSGA-II (výchozí: 50)
        evol_pop: Velikost evoluční populace (výchozí: 100)
    
    Návratové hodnoty:
        final_X: (N, params) Pareto-optimální parametry masky
        final_F: (N, 2) Pareto-optimální hodnoty cílových funkcí
        full_history: Seznam snímků populace pro animaci
    """
    from pymoo.core.problem import Problem
    from pymoo.core.callback import Callback
    from pymoo.algorithms.moo.nsga2 import NSGA2
    from pymoo.optimize import minimize
    from pymoo.operators.crossover.sbx import SBX
    from pymoo.operators.mutation.pm import PM
    
    # ========================================================================
    # Fáze 1: Gradientní seeding
    # ========================================================================
    print("=" * 60)
    print("FÁZE 1: GRADIENTNÍ SEEDING")
    print(f"Populace: {seed_pop}, Kroků: {seed_steps}")
    print("=" * 60)
    
    manager = ParetoManager(encoder, pop_size=seed_pop, learning_rate=0.05)
    manager.calculate_normalization()
    
    phase1_history = []
    
    for step in range(1, seed_steps + 1):
        manager.optimize_step()
        
        # Záznam historie každých 5 kroků (pro animaci)
        if step % 5 == 0:
            vis, aud = manager.get_current_front()
            phase1_history.append(np.column_stack([vis, aud]))
        
        if step % 50 == 0:
            vis, aud = manager.get_current_front()
            print(f"  Krok {step:4d}: Rozsah Vis [{vis.min():.3f}, {vis.max():.3f}], "
                  f"Rozsah Aud [{aud.min():.3f}, {aud.max():.3f}]")
    
    # Extrakce počátečních řešení (seeds)
    with torch.no_grad():
        seed_masks = torch.sigmoid(manager.mask_logits).cpu().numpy()
    
    print(f"\nVygenerováno {len(seed_masks)} počátečních řešení")
    
    # ========================================================================
    # Fáze 2: Evoluční expanze (NSGA-II)
    # ========================================================================
    print("\n" + "=" * 60)
    print("FÁZE 2: EVOLUČNÍ ALGORITMUS")
    print(f"Populace: {evol_pop}, Generací: {n_gen}")
    print("=" * 60)
    
    class MOSSProblem(Problem):
        def __init__(self, enc, scale_vis, scale_aud):
            self.enc = enc
            self.scale_vis = scale_vis
            self.scale_aud = scale_aud
            super().__init__(
                n_var=enc.grid_height * enc.grid_width,
                n_obj=2,
                xl=0.0,
                xu=1.0
            )
        
        def _evaluate(self, x, out, *args, **kwargs):
            with torch.no_grad():
                masks = torch.from_numpy(x).float()
                _, mixed_mag = self.enc(masks, return_wav=False)  # proxy automaticky
                
                vis = calc_visual_loss(mixed_mag, self.enc.image_mag_ref).numpy()
                aud = calc_audio_mag_loss(mixed_mag, self.enc.audio_mag).numpy()
                
                vis = vis * self.scale_vis
                aud = aud * self.scale_aud
                
                out['F'] = np.column_stack([vis, aud])
    
    class HistoryCallback(Callback):
        def __init__(self):
            super().__init__()
            self.history = []
        
        def notify(self, algorithm):
            # Záznam přeživší populace
            self.history.append(algorithm.pop.get('F').copy())
    
    problem = MOSSProblem(encoder, manager.scale_vis, manager.scale_aud)
    
    # Inicializace pomocí seeds + náhodných hodnot
    n_random = evol_pop - len(seed_masks)
    X_init = np.vstack([
        seed_masks,
        np.random.rand(n_random, problem.n_var)
    ])
    
    algorithm = NSGA2(
        pop_size=evol_pop,
        sampling=X_init,
        crossover=SBX(eta=15, prob=0.9),
        mutation=PM(eta=20),
        eliminate_duplicates=True
    )
    
    callback = HistoryCallback()
    
    result = minimize(
        problem,
        algorithm,
        ('n_gen', n_gen),
        callback=callback,
        verbose=True
    )
    
    full_history = phase1_history + callback.history
    
    print(f"\n✅ Nalezeno {len(result.F)} Pareto-optimálních řešení")
    
    return result.X, result.F, full_history

# Spuštění plné Paretovy optimalizace
start = time.time()
pareto_X, pareto_F, pareto_history = run_pareto_optimization(encoder)
print(f"\nCelkový čas: {time.time() - start:.1f}s")

# Vizualizace evoluce
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Finální Paretova fronta
axes[0].scatter(
    pareto_F[:, 0], pareto_F[:, 1],
    c='#4ade80', s=60, edgecolors='white', linewidths=1,
    label='Finální Paretova fronta', zorder=10
)

# Seeds pro referenci
if pareto_history:
    seeds = pareto_history[len(pareto_history) // 4]
    axes[0].scatter(
        seeds[:, 0], seeds[:, 1],
        c='purple', s=40, alpha=0.3, label='Seeding'
    )

axes[0].set_xlabel('Vizuální ztráta (normalizovaná) ↓')
axes[0].set_ylabel('Audio ztráta (normalizovaná) ↓')
axes[0].set_title('Paretova hranice: Vizuální vs Audio kvalita')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Evoluce v čase
n_hist = len(pareto_history)
phase1_len = PARETO_SEED_STEPS // 5  # Historie zaznamenaná každých 5 kroků

for i, frame in enumerate(pareto_history):
    if i < phase1_len:
        color = 'purple'
        alpha = 0.1 + 0.3 * (i / phase1_len)
    else:
        color = '#4ade80'
        alpha = 0.2 + 0.6 * ((i - phase1_len) / (n_hist - phase1_len))
    
    axes[1].scatter(frame[:, 0], frame[:, 1], c=color, alpha=alpha, s=20)

axes[1].set_xlabel('Vizuální ztráta (normalizovaná) ↓')
axes[1].set_ylabel('Audio ztráta (normalizovaná) ↓')
axes[1].set_title('Evoluce v čase (fialová=nasazování, zelená=evoluce)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Animovaný vývoj

Tato animace ukazuje proces optimalizace s:
- **Fáze 1 (Fialová)**: Gradientní nasazování s hladkými přechody bodů
- **Fáze 2 (Zelená)**: NSGA-II evoluce se stopami mizení

In [ ]:
from scipy.spatial.distance import cdist
from matplotlib.animation import FuncAnimation

def create_pareto_animation(history, phase1_frames=40):
    """
    Vytvoří animovanou vizualizaci Pareto optimalizace.
    Odpovídá stylu animace frontend/backend (světlý režim).
    """
    if not history or len(history) == 0:
        print("Žádná historie k animaci")
        return None
    
    # Nastavení grafu (světlá verze)
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_title("Animace Pareto optimalizace", fontsize=14, pad=10)
    ax.set_xlabel("Vizuální ztráta (normalizovaná) ↓")
    ax.set_ylabel("Audio ztráta (normalizovaná) ↓")
    ax.grid(True, alpha=0.3)
    
    # Nalezení globálních mezí
    all_points = np.vstack(history)
    min_x, max_x = all_points[:, 0].min(), all_points[:, 0].max()
    min_y, max_y = all_points[:, 1].min(), all_points[:, 1].max()
    
    pad_x = (max_x - min_x) * 0.1 if max_x > min_x else 0.1
    pad_y = (max_y - min_y) * 0.1 if max_y > min_y else 0.1
    
    ax.set_xlim(min_x - pad_x, max_x + pad_x)
    ax.set_ylim(min_y - pad_y, max_y + pad_y)
    
    # Bodový graf pro aktuální snímek
    scat = ax.scatter([], [], c='#a855f7', alpha=0.8, s=60, 
                      edgecolors='white', linewidths=0.5)
    
    # Bodové grafy stop pro efekt doznívání
    trail_depth = 5
    trail_scats = [ax.scatter([], [], c='#4ade80', alpha=0, s=40) 
                   for _ in range(trail_depth)]
    
    text = ax.text(0.02, 0.98, '', transform=ax.transAxes, fontsize=10, va='top')
    
    prev_positions = [None]  # Použití seznamu pro umožnění mutace v uzávěru
    
    def match_points(prev, curr):
        """Přiřazení bodů mezi snímky pro hladký přechod."""
        if prev is None or len(prev) != len(curr):
            return curr
        dist = cdist(prev, curr)
        matched = np.zeros_like(curr)
        used = set()
        for i in range(len(prev)):
            dists = dist[i].copy()
            dists[list(used)] = np.inf
            j = np.argmin(dists)
            matched[i] = curr[j]
            used.add(j)
        return matched
    
    def update(frame):
        if frame >= len(history):
            return (scat, text, *trail_scats)
        
        data = history[frame]
        
        # Hladký přechod během fáze nasazování
        if frame < phase1_frames:
            data = match_points(prev_positions[0], data)
        prev_positions[0] = data.copy()
        
        scat.set_offsets(data)
        
        # Barvení podle fáze
        if frame < phase1_frames:
            scat.set_facecolors('#a855f7')  # Fialová pro nasazování
            text.set_text(f'Fáze 1: Gradientní seeding (Krok {frame * 5})')
            for ts in trail_scats:
                ts.set_offsets(np.empty((0, 2)))
        else:
            scat.set_facecolors('#4ade80')  # Zelená pro evoluci
            gen = frame - phase1_frames
            text.set_text(f'Fáze 2: Evoluční (Gen {gen})')
            
            # Aktualizace stop s efektem doznívání
            for i, ts in enumerate(trail_scats):
                trail_frame = frame - (i + 1)
                if trail_frame >= phase1_frames and trail_frame < len(history):
                    ts.set_offsets(history[trail_frame])
                    alpha = 0.3 * (1 - (i / trail_depth))
                    ts.set_alpha(alpha)
                else:
                    ts.set_offsets(np.empty((0, 2)))
        
        return (scat, text, *trail_scats)
    
    ani = FuncAnimation(fig, update, frames=len(history), interval=100, blit=True)
    plt.close(fig)  # Zabránění statickému zobrazení
    
    return ani

# Vytvoření a zobrazení animace
phase1_len = PARETO_SEED_STEPS // 5
ani = create_pareto_animation(pareto_history, phase1_frames=phase1_len)
if ani:
    display(HTML(ani.to_jshtml()))

## 9. Závěrečná analýza a Shrnutí

V tomto notebooku jsme úspěšně implementovali end-to-end systém pro syntézu spektrogramů nesoucích zvuk a obraz zároveň. Hlavním výsledkem je relativně rychlé nalezení Paretovy fronty pro vícekriteriální optimalizaci s velmi vysokou dimenzí prostoru parametrů.

### Designová rozhodnutí v kontextu fyziky

*   **Čas vs. Frekvence:** Volba $N_{FFT}=1024$ při 16kHz je přímým důsledkem snahy vyvážit časovou lokalizaci (pro rytmus) a frekvenční rozlišení (pro detaily obrazu).
*   **Fázová invariance:** Rozhodnutí ignorovat vizuální fázi a vnutit fázi audia je to, co činí výsledek poslouchatelným. Předpokládáme, že sémantika zvuku je nesena časovým uspořádáním fází.

### Citace a Odkazy

*   **NSGA-II:** Deb, K., Pratap, A., Agarwal, S., & Meyarivan, T. (2002). A fast and elitist multiobjective genetic algorithm: NSGA-II. *IEEE Transactions on Evolutionary Computation*.
*   **STFT Reconstruction:** Griffin, D., & Lim, J. (1984). Signal estimation from modified short-time Fourier transform. *IEEE Transactions on ASSP*.
*   **Pareto Optimality:** Pareto, V. (1896). *Cours d'économie politique*.
*   **ADAM:** Kingma, D. P., & Ba, J. (2014). Adam: A method for stochastic optimization. *arXiv preprint arXiv:1412.6980*.
*   **L1 Loss in Image Synthesis:** Zhao, H., Gallo, O., Frosio, I., & Kautz, J. (2016). Loss functions for image restoration with neural networks. *IEEE Transactions on Computational Imaging*, 3(1), 47-57.
*   **Phase Reconstruction:** Allen, J. B. (1977). Short term spectral analysis, synthesis, and modification by discrete Fourier transform. *IEEE Transactions on Acoustics, Speech, and Signal Processing*, 25(3), 235-238.
*   **Psychoacoustics:** Fastl, H., & Zwicker, E. (2007). *Psychoacoustics: Facts and models*. Springer.
*   **Tacotron and Log L1 loss:** Wang, Y., et al. (2017). "Tacotron: Towards end-to-end speech synthesis." *Interspeech*.

Tento systém demonstruje, že i s relativně jednoduchými stavebními kameny (STFT, lineární interpolace) a robustní optimalizací lze dosáhnout netriviálních a možná umělecky zajímavých výsledků.

In [ ]:
# Průzkum Paretovy fronty

sorted_idx = np.argsort(pareto_F[:, 0])

# Výběr 5 vzorků podél hranice
n_samples = min(5, len(sorted_idx))
sample_indices = [sorted_idx[int(i * (len(sorted_idx) // 3) / (n_samples - 1))] 
                  for i in range(n_samples)]

fig, axes = plt.subplots(2, n_samples, figsize=(4 * n_samples, 8))

for col, idx in enumerate(sample_indices):
    mask = torch.from_numpy(pareto_X[idx:idx+1]).float()
    wav, mixed_mag = encoder(mask, return_wav=True)
    
    # Maska
    axes[0, col].imshow(
        mask.view(GRID_HEIGHT, GRID_WIDTH).numpy(),
        cmap='gray', vmin=0, vmax=1
    )
    axes[0, col].set_title(f'V:{pareto_F[idx,0]:.3f}, A:{pareto_F[idx,1]:.3f}')
    axes[0, col].axis('off')
    
    # Spektrogram (škálování odpovídající frontendu)
    plot_spectrogram(axes[1, col], mixed_mag[0])
    axes[1, col].axis('off')

plt.suptitle('Vzorky napříč Paretovou frontou (Vlevo=Nejlepší vizuální, Vpravo=Nejlepší audio)')
plt.tight_layout()
plt.show()

# Poslech vzorků
for i, idx in enumerate(sample_indices):
    mask = torch.from_numpy(pareto_X[idx:idx+1]).float()
    wav, _ = encoder(mask, return_wav=True)
    print(f"\n🎵 Vzorek {i+1}: Vizuální={pareto_F[idx,0]:.3f}, Audio={pareto_F[idx,1]:.3f}")
    display(Audio(wav[0].numpy(), rate=SAMPLE_RATE))

*Vytvořil s 💜 [Vojtěch Kuchař](https://github.com/kuchar-one)*